In [ ]:
import datetime
import json
import os
import re
from typing import List, Union

import dotenv
import pandas as pd
from langchain.agents import (
    AgentExecutor,
    AgentOutputParser,
    LLMSingleActionAgent,
    Tool,
    initialize_agent,
)
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatAnthropic
from langchain.prompts import BaseChatPromptTemplate
from langchain.schema import AgentAction, AgentFinish, HumanMessage
from langchain.utilities import PythonREPL, WikipediaAPIWrapper

from redbox.llm.llm_base import LLMHandler

In [ ]:
if os.path.basename(os.getcwd()) != "10ds-ai-redbox":
    os.chdir("..")
    print(os.getcwd())

In [ ]:
dotenv.load_dotenv(".env")
# Grab it as a dictionary too for convenience
ENV = dotenv.dotenv_values(".env")

model_params = {"max_tokens": 4096, "temperature": 0.2}

llm = ChatAnthropic(
    anthropic_api_key=ENV["ANTHROPIC_API_KEY"],
    max_tokens=model_params["max_tokens"],
    temperature=model_params["temperature"],
    streaming=True,
)

llm_instance = LLMHandler(llm, user_uuid="dev")

url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSf5yPnrU5Z_iIP7L3op6deIm5PeTrsr7RYgc82oBxzh4ZPoIa-JhTNr4YKECQA_AxDM616oVPPjQbr/pub?output=csv"
acronym_table = pd.read_csv(url)

In [ ]:
def acronym(acronym=""):
    return json.dumps(
        acronym_table.loc[acronym_table["Acronym"] == acronym].to_dict("records"),
        indent=4,
        ensure_ascii=False,
    )


def todays_date_and_time(_):
    return datetime.datetime.now().isoformat(timespec="seconds")


# Tools

python_repl = PythonREPL()
python_repl_tool = Tool(
    name="Python REPL",
    func=python_repl.run,
    description="""Useful for when you need to use python to answer a question. \
You should input python code. You MUST USE the datetime library for any date or time related questions.""",
)

wikipedia = WikipediaAPIWrapper()
wikipedia_tool = Tool(
    name="Wikipedia",
    func=wikipedia.run,
    description="Useful for when you need to look up a topic that isn't defined in the provided sources",
)

acronym_tool = Tool(
    name="Acronym Tool",
    func=acronym,
    description="A definitive look up tool for any Civil Service Acronym and Office abbreviation.",
)

todays_date_and_time_tool = Tool(
    name="Today's Date and Time Tool",
    func=todays_date_and_time,
    description="A tool to get today's date and time.",
)

In [ ]:
print(acronym_tool.run(""))

In [ ]:
tools = [
    python_repl_tool,
    # wikipedia_tool,
    acronym_tool,
    todays_date_and_time_tool,
]

In [ ]:
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=5,
    early_stopping_method="generate",
)

In [ ]:
memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=3, return_messages=True
)

In [ ]:
retriever_tool = create_retriever_tool(
    llm_instance.vector_store.as_retriever(search_kwargs={"k": 10}),
    "Search",
    "This is the primary information search tool for documents uploaded by the user.",
)
retriever_tools = [retriever_tool] + tools

In [ ]:
conversational_agent = initialize_agent(
    agent="chat-conversational-react-description",
    tools=retriever_tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method="generate",
    memory=memory,
)

In [ ]:
conversational_agent("When did Rishi Sunak announce the AI Safety Institute?")

In [ ]:
conversational_agent("How many days ago was that?")

In [ ]:
template = """Your name is Redbox Copilot. You help people in UK Government answer questions and discover information.
Be objective in your analysis and answers.
Use the Redbox search tools to get source information.
When working with time ALWAYS use the tools provided to calulate the time difference.
Complete the objective given as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

These were previous tasks you completed:



Begin!

Question: {input}
{agent_scratchpad}"""


# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join(
            [f"{tool.name}: {tool.description}" for tool in self.tools]
        )
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]


prompt = CustomPromptTemplate(
    template=template,
    tools=retriever_tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"],
)


class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(
            tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output
        )


output_parser = CustomOutputParser()

In [ ]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

tool_names = [tool.name for tool in retriever_tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names,
)

In [ ]:
memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=3, return_messages=True
)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=retriever_tools, verbose=True, memory=memory
)

In [ ]:
agent_executor.run("How many years until Artificial General Intelligence?")